# 31-仿射变换（Afine Transformations）——倾斜

1. 使用仿射变换，输出（1）那样的$x$轴倾斜$30$度的图像（$t_x=30$），这种变换被称为X-sharing。
2. 使用仿射变换，输出（2）那样的y轴倾斜$30$度的图像（$t_y=30$），这种变换被称为Y-sharing。
3. 使用仿射变换，输出（3）那样的$x$轴、$y$轴都倾斜$30$度的图像($t_x = 30$，$t_y = 30$)。
4. 原图像的大小为$h\ w$，使用下面各式进行仿射变换：

X-sharing $$ a=\frac{t_x}{h}\ \left[ \begin{matrix} x'\ y'\ 1 \end{matrix} \right]=\left[ \begin{matrix} 1&a&t_x\ 0&1&t_y\ 0&0&1 \end{matrix} \right]\ \left[ \begin{matrix} x\ y\ 1 \end{matrix} \right] $$

Y-sharing $$ a=\frac{t_y}{w}\ \left[ \begin{matrix} x'\ y'\ 1 \end{matrix} \right]=\left[ \begin{matrix} 1&0&t_x\ a&1&t_y\ 0&0&1 \end{matrix} \right]\ \left[ \begin{matrix} x\ y\ 1 \end{matrix} \right] $$

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread('../picture/chans.png').astype(np.float32)

In [3]:
def affine(img, dx=30, dy=30):
    # get shape
    H, W, C = img.shape

    # Affine hyper parameters
    a = 1.
    b = dx / H
    c = dy / W
    d = 1.
    tx = 0.
    ty = 0.

    # prepare temporary
    _img = np.zeros((H+2, W+2, C), dtype=np.float32)

    # insert image to center of temporary
    _img[1:H+1, 1:W+1] = img

    # prepare affine image temporary
    H_new = np.ceil(dy + H).astype(np.int)
    W_new = np.ceil(dx + W).astype(np.int)
    out = np.zeros((H_new, W_new, C), dtype=np.float32)

    # preprare assigned index
    x_new = np.tile(np.arange(W_new), (H_new, 1))
    y_new = np.arange(H_new).repeat(W_new).reshape(H_new, -1)

    # prepare inverse matrix for affine
    adbc = a * d - b * c
    x = np.round((d * x_new  - b * y_new) / adbc).astype(np.int) - tx + 1
    y = np.round((-c * x_new + a * y_new) / adbc).astype(np.int) - ty + 1

    x = np.minimum(np.maximum(x, 0), W+1).astype(np.int)
    y = np.minimum(np.maximum(y, 0), H+1).astype(np.int)

    # assign value from original to affine image
    out[y_new, x_new] = _img[y, x]
    out = out.astype(np.uint8)

    return out

In [4]:
img_af4 = affine(img, dx=30, dy=30)

In [5]:
cv2.imwrite('../picture/chan_result31_Afine Transformations.jpg', img_af4)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_af4)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 32-傅立叶变换（Fourier Transform）

使用离散二维傅立叶变换（Discrete Fourier Transformation），将灰度化的imori.jpg表示为频谱图。然后用二维离散傅立叶逆变换将图像复原。

二维离散傅立叶变换是傅立叶变换在图像处理上的应用方法。通常傅立叶变换用于分离模拟信号或音频等连续一维信号的频率。但是，数字图像使用$[0,255]$范围内的离散值表示，并且图像使用$H\times W$的二维矩阵表示，所以在这里使用二维离散傅立叶变换。

二维离散傅立叶变换使用下式计算，其中$I$表示输入图像： $$ G(k,l)=\frac{1}{H\ W}\ \sum\limits_{y=0}^{H-1}\ \sum\limits_{x=0}^{W-1}\ I(x,y)\ e^{-2\ \pi\ j\ (\frac{k\ x}{W}+\frac{l\ y}{H})} $$ 在这里让图像灰度化后，再进行离散二维傅立叶变换。

频谱图为了能表示复数$G$，所以图上所画长度为$G$的绝对值。这回的图像表示时，请将频谱图缩放至$[0,255]$范围。

二维离散傅立叶逆变换从频率分量$G$按照下式复原图像： $$ I(x,y)=\frac{1}{H\ W}\ \sum\limits_{l=0}^{H-1}\ \sum\limits_{k=0}^{W-1}\ G(l,k)\ e^{2\ \pi\ j\ (\frac{k\ x}{W}+\frac{l\ y}{H})} $$

上式中$\exp(j)$是个复数，实际编程的时候请务必使用下式中的绝对值形态^1：

如果只是简单地使用for语句的话，计算量达到$128^4$，十分耗时。如果善用NumPy的化，则可以减少计算量（答案中已经减少到$128^2$）。

In [21]:
# DFT hyper-parameters
K, L = 64, 64
channel = 3

In [22]:
# DFT
def dft(img):
    H, W, C = img.shape

    G = np.zeros((L, K, channel), dtype=np.complex)
    # np.complex创建一个值为 real+imag*j的复数或者转化为一个字符串或数为复数？？

    # 准备对应于原始图像位置的处理过的索引
    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # dft
    for c in range(channel):
            for l in range(L):
                for k in range(K):
                    G[l, k, c] = np.sum(img[..., c] * np.exp(-2j * np.pi * (x * k / K + y * l / L))) / np.sqrt(K * L)

    return G

In [24]:
# IDFT
def idft(G):
    H, W, _ = G.shape
    out = np.zeros((H, W, channel), dtype=np.float32)

    # 准备对应于原始图像位置的处理过的索引
    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # idft
    for c in range(channel):
            for l in range(H):
                for k in range(W):
                    out[l, k, c] = np.abs(np.sum(G[..., c] * np.exp(2j * np.pi * (x * k / W + y * l / H)))) / np.sqrt(W * H)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

In [19]:
img_gray = cv2.imread('../picture/chan_small.jpg').astype(np.float32)

In [20]:
img_gray.shape

(64, 64, 3)

In [25]:
G = dft(img_gray)

In [26]:
ps = (np.abs(G) / np.abs(G).max() * 255).astype(np.uint8)
cv2.imwrite("../picture/chan_result32_Frequency Domain.jpg", ps)

True

In [27]:
I = idft(G)

In [28]:
cv2.imwrite('../picture/chan_result32_Fourier Transform.jpg', I)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", I)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 33-傅立叶变换——低通滤波

将图像灰度化之后进行傅立叶变换并进行低通滤波，之后再用傅立叶逆变换复原

通过离散傅立叶变换得到的频率在左上、右上、左下、右下等地方频率较低，在中心位置频率较高。

在图像中，高频成分指的是颜色改变的地方（噪声或者轮廓等），低频成分指的是颜色不怎么改变的部分（比如落日的渐变）。在这里，使用去除高频成分，保留低频成分的低通滤波器

在这里，假设从低频的中心到高频的距离为$r$，我们保留$0.5\ r$​的低频分量。

In [29]:
# LPF
def lpf(G, ratio=0.5):
    H, W, C = G.shape

    _G = np.zeros_like(G)
    _G[:H//2, :W//2] = G[H//2:, W//2:]
    _G[:H//2, W//2:] = G[H//2:, :W//2]
    _G[H//2:, :W//2] = G[:H//2, W//2:]
    _G[H//2:, W//2:] = G[:H//2, :W//2]

    x = np.tile(np.arange(W), (H, 1))
    y = np.arange(H).repeat(W).reshape(H, -1)

    # 滤波器
    _x = x - W // 2
    _y = y - H // 2
    r = np.sqrt(_x ** 2 + _y ** 2)
    mask = np.ones((H, W), dtype=np.float32)
    mask[r > (W // 2 * ratio)] = 0

    mask = np.repeat(mask, channel).reshape(H, W, channel)

    # filtering
    _G *= mask

    G[:H//2, :W//2] = _G[H//2:, W//2:]
    G[:H//2, W//2:] = _G[H//2:, :W//2]
    G[H//2:, :W//2] = _G[:H//2, W//2:]
    G[H//2:, W//2:] = _G[:H//2, :W//2]

    return G

In [33]:
# 将傅里叶变换后图像进行低通滤波
img_ = lpf(G)

In [34]:
img_lpf = idft(img_)

In [35]:
cv2.imwrite('../picture/chan_result33_lpf.jpg', img_lpf)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_lpf)
cv2.waitKey(0)
cv2.destroyAllWindows()